In [3]:
import pandas as pd
import os
import re
import json

def span_column(df, column, apply_func):
    for name, values in df[column].apply(apply_func).iteritems():
        df[column + str(name)] = values
    return df

def preprocess(df):
    df = df[['beginTime', 'imageCloudage', 'imageResolution', 'imageSatelliteType', 'range']]
    df['polygon'] = df['range'].apply(lambda s: json.loads(s)['coordinates'][0][0])
    del df['range']
    return df

def get_merged_df():
    out_file = os.path.join('..', 'data', 'interim', 'merged.csv')
    if os.path.exists(out_file):
        merged_df = pd.read_csv(out_file)
        print('exist file loaded')
    else:
        dfs = []
        for dirpath, dirnames, filenames in os.walk('../data/raw'):
            for filename in filenames:
                df = pd.read_csv(os.path.join(dirpath, filename))
                df = preprocess(df)
                dfs.append(df)
        merged_df = pd.concat(dfs)
        merged_df.to_csv(out_file)
    return merged_df

merged_df = get_merged_df()

merged_df.head()

C:\Users\enhuiz\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,beginTime,imageCloudage,imageResolution,imageSatelliteType,polygon
0,2017-05-01 00:00:00,100.0,{250.00},Terra,"[[105.6498566, 72.7449417], [105.6498566, 49.8..."
1,2017-05-01 00:00:00,100.0,{250.00},Terra,"[[80.4741516, 50.8296852], [80.4741516, 29.337..."
2,2017-05-01 00:00:00,100.0,{250.00},Terra,"[[55.1295471, 46.8265228], [55.1295471, 25.451..."
3,2017-05-01 00:00:00,100.0,{250.00},Terra,"[[77.6394348, 32.8995628], [77.6394348, 11.802..."
4,2017-05-01 00:00:00,100.0,{250.00},Terra,"[[57.142128, 64.7271881], [57.142128, 42.56574..."


In [4]:
import json

def get_nonfree_price_df():
    price_map = {
        'HJ1B-CCD': '0.00',
        'sentinel2A-S2A': '0.00',
        'ZY02C-HRC-{2.36}': '1800.00', #v
        'SPARK01-HS': '0.00',
        'GF4-PMS': '0.00',
        'HJ1B-IRS': '0.00',
        'HJ1A-HSI': '0.00',
        'PL-': '0.00',
        'GF2-PMS-{0.8,3.2}': '9000.00', #v
        'Aqua-MYD02': '0.00',
        'ZY3-NAD-{2.10}': '1500.00', #v
        'GF4-IRS': '0.00',
        'PL-PS0': '0.00',
        'PL-PS1': '0.00',
        'PL-PS2-{3.00}': '700.00', #v
        'ZY3-MUX-{5.00}': '1000.00', #v
        #'ZY3-BWD': '2300.00',
        #'ZY3-DLC': '3000.00',
        #'ZY3-FWD': '2300.00',
        'Terra-MOD02': '0.00',
        'HJ1A-CCD': '0.00',
        'GF1-PMS-{2.00,8.00}': '1500.00', #v
        'ZY02C-PMS-{5.00,10.00}': '2500.00', #v
        'SPARK02-HS': '0.00',
        'GF1-WFV': '0.00',
        'GF3-SAR': '0.00'
    }

    def parse_dict(s):
        ss = s.split('-')
        ss += (3 - len(ss)) * ['']
        return {'imageSatelliteType': ss[0], 'imageSensorType': ss[1], 'imageResolution': ss[2]}

    price_df = pd.DataFrame([{**parse_dict(k), 'imagePrice':price} 
                             for k, price in price_map.items()])[['imageSatelliteType', 
                                                                  'imageResolution', 
                                                                  'imageSensorType', 
                                                                  'imagePrice']]

    nonfree_price_df = price_df[price_df['imagePrice'].astype('float') > 0].reset_index(drop=True)
    
    return nonfree_price_df

nonfree_price_df = get_nonfree_price_df()

In [8]:
def get_final_df(merged_df, nonfree_price_df):

    final_df = pd.merge(merged_df, nonfree_price_df, on=['imageSatelliteType', 'imageResolution'])

    final_df.columns = ['Time', 'Cloudage', 'Resolution', 'Satellite', 'Polygon', 'Sensor', 'Price']
    
    return final_df

final_df = get_final_df(merged_df, nonfree_price_df)

final_df.head()

,Time,Cloudage,Resolution,Satellite,Polygon,Sensor,Price
0,2017-05-01 04:14:19,20.0,"{2.00,8.00}",GF1,"[[112.207, 45.0734], [112.084, 44.7462], [112....",PMS,1500.00
1,2017-05-01 04:14:23,18.0,"{2.00,8.00}",GF1,"[[112.103, 44.7975], [111.981, 44.4703], [112....",PMS,1500.00
2,2017-05-01 04:16:31,34.0,"{2.00,8.00}",GF1,"[[109.487, 37.0474], [109.388, 36.7198], [109....",PMS,1500.00
3,2017-05-01 04:16:40,34.0,"{2.00,8.00}",GF1,"[[108.941, 36.5636], [108.846, 36.2402], [109....",PMS,1500.00
4,2017-05-01 04:16:45,17.0,"{2.00,8.00}",GF1,"[[108.861, 36.2853], [108.77, 35.9759], [109.1...",PMS,1500.00


In [7]:
def to_final_csv(final_df):
    out_file = os.path.join('..', 'data', 'processed', 'rsmeta.csv')
    final_df.to_csv(out_file)

to_final_csv(final_df)

In [9]:
import sqlite3

def to_final_db(final_df):
    out_file = os.path.join('..', 'data', 'processed', 'rsmeta.db')
    con = sqlite3.connect(out_file)

    sql_df = final_df.copy()
    sql_df['Polygon'] = sql_df['Polygon'].apply(str)

    sql_df.to_sql("rsmeta", con, if_exists="replace")

    con.close()

to_final_db(final_df)